### JSON Mode issues:
1. the key names in the returned JSON are not fixed
2. the values are also uncertain, can get a string where you are expecting integer

In [4]:
import openai
import pandas as pd
import json
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
import re

In [5]:
input_data = pd.read_excel('rpa pricing extraction.xlsx')

In [6]:
output_data = input_data.copy()

In [7]:
output_data["Max Discount"]=None
output_data["IBD Bank Name/UPI"]=None
output_data["Min Swipe"]=None
output_data["Bump up"]=None
output_data["VPC(Amazon)"]=None
output_data["SuperCoins(Flipkart)"]=None
output_data.drop('BUP', axis=1, inplace=True)
output_data.drop('VPC', axis=1, inplace=True)
output_data.drop('SuperCoins', axis=1, inplace=True)
output_data

,Market Place,FSN /ASIN,Description,MRP,Current Price,Bank Offers,URL,Payload,API Status,Max Discount,IBD Bank Name/UPI,Min Swipe,Bump up,VPC(Amazon),SuperCoins(Flipkart)
0,Amazon,B0BZ48VZMR,"realme narzo N55 (Prime Black, 4GB+64GB) 33W S...","₹12,999","10,999",1.Additional Flat INR 250 Instant Discount on ...,https://www.amazon.in/dp/B0BZ48VZMR,NaN,NaN,None,None,None,None,None,None
1,Amazon,B0BZ48H8JX,"realme narzo N55 (Prime Black, 6GB+128GB) 33W ...","₹14,999","12,999",NaN,https://www.amazon.in/dp/B0BZ48H8JX,NaN,NaN,None,None,None,None,None,None
2,Amazon,B0BY8MCQ9S,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB...",0,"19,999",NaN,https://www.amazon.in/dp/B0BY8MCQ9S,NaN,NaN,None,None,None,None,None,None
3,Amazon,B0BY8L3RZ6,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB...",0,"21,999",NaN,https://www.amazon.in/dp/B0BY8L3RZ6,NaN,NaN,None,None,None,None,None,None
4,Amazon,B0C788SHHC,"realme narzo 60 5G (Mars Orange,8GB+256GB) | 9...","₹20,999","18,499",1.10% Instant Discount up to INR 750 on SBI Cr...,https://www.amazon.in/dp/B0C788SHHC,NaN,NaN,None,None,None,None,None,None
5,Amazon,B0C787P8BV,"realme narzo 60 5G (Mars Orange,8GB+256GB) | 9...","₹20,999","18,499",1.10% Instant Discount up to INR 750 on SBI Cr...,https://www.amazon.in/dp/B0C787P8BV,NaN,NaN,None,None,None,None,None,None
6,Amazon,B0C787GXXR,"realme narzo 60 Pro (Cosmic Black,12GB+1TB) Ul...","₹32,999","26,999",1.10% Instant Discount up to INR 750 on SBI Cr...,https://www.amazon.in/dp/B0C787GXXR,NaN,NaN,None,None,None,None,None,None
7,Amazon,B0C788GH1F,"realme narzo 60 Pro (Mars Orange,12GB+256GB) U...","₹28,999","23,999",1.10% Instant Discount up to INR 750 on SBI Cr...,https://www.amazon.in/dp/B0C788GH1F,NaN,NaN,None,None,None,None,None,None
8,Amazon,B0C788T92F,"realme narzo 60 Pro (Mars Orange,8GB+128GB) Me...","₹26,999","21,249",1.10% Instant Discount up to INR 750 on SBI Cr...,https://www.amazon.in/dp/B0C788T92F,NaN,NaN,None,None,None,None,None,None
9,Amazon,B0C9QPJY5R,"Oneplus Nord CE 3 5G (Grey Shimmer, 8GB RAM, 1...",0,"26,999",1.Additional Flat INR 1250 Instant Discount on...,https://www.amazon.in/dp/B0C9QPJY5R,NaN,NaN,None,None,None,None,None,None


In [8]:
def create_content(price, offers):
    content=f"The Price of the SKU is {price}\n\nOffers:\n\n{offers}\n\nWhich is the best Offer?"
    return content

In [9]:
GPT_MODEL='gpt-3.5-turbo-1106'
#GPT_MODEL='gpt-4-0613'

In [10]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages, "response_format":{ "type": "json_object" }}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [11]:
def extract_discount_details(data):
    try:
        # Parse the JSON string in the 'arguments' field
        arguments_json = json.loads(data['content'])
        print("ARGUMENTS_JSON", arguments_json)
        data_list = list(arguments_json.items())

        # Extract values
        discount_bank = data_list[0][1]
        discount_amount = data_list[1][1]
        discount_minimum_purchase = data_list[2][1]

        return discount_bank, discount_amount, discount_minimum_purchase

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error extracting discount details: {e}")
        return None, None, None

In [12]:
def extract_bup_details(bup_details):
    bup=0
    bup = re.sub(r'[^\d.]', '', bup_details)
    return bup
    

In [13]:
def extract_vpc_details(vpc_details):
    vpc=0
    vpc = re.sub(r'[^\d.]', '', vpc_details)
    return vpc

In [14]:
def extract_supercoins_details(supercoins_details):
    supercoins=0
    #supercoins = re.sub(r'[^\d.]', '', supercoins_details)
    supercoins=re.findall(r'\d+', supercoins_details)[0] if re.findall(r'\d+', supercoins_details) else None
    return supercoins

In [16]:
len(output_data)

19

In [17]:
messages = [
    {"role": "system", "content": "You are a helpful assistant designed to output JSON. You help in identifying the offer with maximum discount. If any offer have more than one discount % always take the higher one. Summarize the best offer at the end by clearly mentioning Maximum discount/cashback, Bank Name & minimum swipe amount on separate lines"},
    {"role": "user", "content": "The Price of SKU is 17999\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n5. Flat ₹3,000 off on HDFC Bank Credit Card EMI Trxns on orders priced ₹50,000 and above\n\nWhich is the best offer?"},
    {"role": "assistant", "content": "Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100 cashback + 500 welcome rewards= 600. so the discount from this offer is 600. From the third offer since the purchase value 17999 is greater than Minimum purchase value of ₹15999, hence the discount amount is applicable & it is equal to ₹500. From the fourth offer the discount = ₹750 since the Minimum pucrhase value of ₹15999 is less than than ₹17999. From the fifth offer the discount is not applicable since sku price (17999) is less than 50000. Comparing the discounts from all offers the maximum discount is ₹900. So maximum discount = ₹900; Bank Name = Amazon Pay ICICI Bank, Minimum Swipe for availing maximum discount = 0"},
    {"role": "user", "content": "The Price of the SKU is 18999\n\nOffers:\n\n1. Flat INR 2000 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n6. ₹100 cashback & ₹2150 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer"},
    {"role": "assistant", "content": "From the first offer since the price value of 18999 is greater than 17999 the discount amount is applicable & it is equal to 2000 . From the second offer since the price value of 18999 is greater than 17999 the discount amount is 2100. From the third offer since the price value of 18999 is less than minimum purchase value of 19999, the discount value is not applicable. From the fourth offer, the discount amount is 5% of the price of the SKU, up to a maximum of 250. Since the price of the SKU is 18999, the discount amount will be 250. Assuming prime member, from the fifth offer, the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of the price of the SKU, which is 18999. This equals 950.From the sixth offer, the discount amount is 100 cashback + 2150 welcome rewards = 2250. Comparing the discounts from the offers, the maximum discount is 2250. So, \nthe maximum discount applicable = 2250\nBank Name = Amazon Pay Later\nMinimum Swipe for availing maximum Discount = 0"},
    {"role": "user", "content": "The Price of the SKU is 18999\n\nOffers:\n\n1. Flat INR 2000 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n6. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer?"},
]

In [18]:
output=chat_completion_request(messages)
print(output.json()['choices'][0]['message'])

{'role': 'assistant', 'content': '{\n  "maximum_discount_cashback": "5% back with Amazon Pay ICICI Bank credit card for Prime members",\n  "bank_name": "Amazon Pay ICICI Bank",\n  "minimum_swipe_amount": "0"\n}'}


In [19]:
response = output.json()['choices'][0]['message']
finish_reason = output.json()['choices'][0]['finish_reason']
print(finish_reason)

stop


In [20]:
bank_name = ""
max_discount=0
min_swipe=0

if finish_reason=='stop':
    bank_name, max_discount, min_swipe = extract_discount_details(response)
    print(bank_name)

ARGUMENTS_JSON {'maximum_discount_cashback': '5% back with Amazon Pay ICICI Bank credit card for Prime members', 'bank_name': 'Amazon Pay ICICI Bank', 'minimum_swipe_amount': '0'}
5% back with Amazon Pay ICICI Bank credit card for Prime members


In [15]:
bank_name

''

In [ ]:
for i in range(len(output_data)):
    print("value of i", i)
    sku_price = output_data['Current Price'][i]
    sku_offers = output_data['Bank Offers'][i]
    content = create_content(sku_price, sku_offers)
    print(content)
    content
    messages = [
    {"role": "system", "content": "You are a helpful assistant designed to output JSON. You help in identifying the offer with maximum discount. If any offer have more than one discount % always take the higher one. Summarize the best offer at the end by clearly mentioning Maximum discount/cashback, Bank Name & minimum swipe amount on separate lines"},
    {"role": "user", "content": "The Price of SKU is 17999\n\nOffers:\n\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n5. Flat ₹3,000 off on HDFC Bank Credit Card EMI Trxns on orders priced ₹50,000 and above\n\nWhich is the best offer?"},
    {"role": "assistant", "content": "Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100 cashback + 500 welcome rewards= 600. so the discount from this offer is 600. From the third offer since the purchase value 17999 is greater than Minimum purchase value of ₹15999, hence the discount amount is applicable & it is equal to ₹500. From the fourth offer the discount = ₹750 since the Minimum pucrhase value of ₹15999 is less than than ₹17999. From the fifth offer the discount is not applicable since sku price (17999) is less than 50000. Comparing the discounts from all offers the maximum discount is ₹900. So maximum discount = ₹900; Bank Name = Amazon Pay ICICI Bank, Minimum Swipe for availing maximum discount = 0"},
    {"role": "user", "content": "The Price of the SKU is 18999\n\nOffers:\n\n1. Flat INR 2000 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n6. ₹100 cashback & ₹2150 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer"},
    {"role": "assistant", "content": "From the first offer since the price value of 18999 is greater than 17999 the discount amount is applicable & it is equal to 2000 . From the second offer since the price value of 18999 is greater than 17999 the discount amount is 2100. From the third offer since the price value of 18999 is less than minimum purchase value of 19999, the discount value is not applicable. From the fourth offer, the discount amount is 5% of the price of the SKU, up to a maximum of 250. Since the price of the SKU is 18999, the discount amount will be 250. Assuming prime member, from the fifth offer, the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of the price of the SKU, which is 18999. This equals 950.From the sixth offer, the discount amount is 100 cashback + 2150 welcome rewards = 2250. Comparing the discounts from the offers, the maximum discount is 2250. So, \nthe maximum discount applicable = 2250\nBank Name = Amazon Pay Later\nMinimum Swipe for availing maximum Discount = 0"},
    {"role": "user", "content": "The Price of the SKU is 18999\n\nOffers:\n\n1. Flat INR 2000 Instant Discount on HDFC Bank Credit CardTxn. Minimum purchase value INR 17999\n2. Flat INR 2100 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit CardTxn. Minimum purchase value INR 17999\n3. Flat INR 2000 Instant Discount on HDFC Bank Debit Card EMI Txn. Minimum purchase value INR 19999\n4. 5% Instant Discount up to INR 250 on HSBC Cashback Card Credit Card Transactions. Minimum purchase value INR 1000\n5. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n6. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later.\n\nWhich is the best Offer?"},
    {"role": "assistant", "content": "From the first offer since the price value of 18999 is greater than 17999 the discount amount is applicable & it is equal to 2000 . From the second offer since the price value of 18999 is greater than 17999 the discount amount is applicable & it is equal to 2100. From the third offer since the price value of 18999 is less than minimum purchase value of 19999, the discount value is not applicable. From the fourth offer, the discount amount is 5% of the price of the SKU, up to a maximum of 250. Since the price of the SKU is 18999, the discount amount will be 250. Assuming prime member, from the fifth offer, the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of the price of the SKU, which is 18999. This equals 950.From the sixth offer, the discount amount is 100 cashback + 500 welcome rewards = 600. Comparing the discounts from the offers, the maximum discount is 2000. So, \nthe maximum discount applicable = 2000\nBank Name = HDFC Bank Credit Card\nMinimum Swipe for availing maximum Discount = 17999"},
    {"role": "user", "content": content}
    ]
    output=chat_completion_request(messages)
    print(output.json()['choices'][0]['message'])
    response = output.json()['choices'][0]['message']
    finish_reason = output.json()['choices'][0]['finish_reason']
    bank_name = ""
    max_discount=0
    min_swipe=0
    print( "FINISH REASON:", finish_reason)
    if finish_reason=='stop':
        bank_name, max_discount, min_swipe = extract_discount_details(response)
    output_data["Max Discount"][i] = max_discount
    output_data['IBD Bank Name/UPI'][i] = bank_name
    output_data['Min Swipe'][i]= min_swipe

    if input_data["BUP"][i] != None and pd.isna(input_data["BUP"][i])==False:
        bup= extract_bup_details(input_data["BUP"][i])
        #print(bup)
        output_data['Bump up'][i]=bup

    if input_data["VPC"][i] != None and pd.isna(input_data["VPC"][i])==False:
        #print("VPC value from input file", input_data["VPC"][i])
        vpc= extract_vpc_details(input_data["VPC"][i])
        output_data['VPC(Amazon)'][i]=vpc

    if input_data["SuperCoins"][i] != None and pd.isna(input_data["SuperCoins"][i])==False:
        #print('supercoins', output_data["SuperCoins(Flipkart)"][i])
        supercoins= extract_supercoins_details(input_data["SuperCoins"][i])
        output_data['SuperCoins(Flipkart)'][i]=supercoins
    output_data.to_excel('output_data.xlsx')

